In [1]:
import os
os.chdir('../')
%pwd

'd:\\Ml'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: Path
    all_schema: dict    

In [5]:
from src.constants import *
from src.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILEPATH,
        params_filepath = PARAMS_FILEPATH,
        schema_filepath = SCHEMA_FILEPATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        

        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self)-> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS             ## from schema.yaml 
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )
        
        return data_validation_config

In [8]:
import pandas as pd

In [9]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                    
            return validation_status
                        
                        
        except Exception as e:
            raise e
        

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-09-14 11:22:57,550: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2023-09-14 11:22:57,551: INFO: __init__: yaml file: params.yaml loaded successfully]
[2023-09-14 11:22:57,553: INFO: __init__: yaml file: schema.yaml loaded successfully]
[2023-09-14 11:22:57,554: INFO: __init__: created directory at: artifacts]
[2023-09-14 11:22:57,555: INFO: __init__: created directory at: artifacts/data-validation]
